<a href="https://colab.research.google.com/github/AlexanderLontke/ssl-remote-sensing/blob/gan%2Fpipeline/notebooks/Classification_Downstream_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Downstream Task 

In [1]:
!pip install ssl_remote_sensing@git+https://github.com/AlexanderLontke/ssl-remote-sensing.git@gan/pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AlexanderLontke/ssl-remote-sensing.git (to revision gan/pipeline) to /tmp/pip-install-fx82imqt/ssl-remote-sensing_5f23d22ea442468c8cc209071be0b5d8
  Running command git clone -q https://github.com/AlexanderLontke/ssl-remote-sensing.git /tmp/pip-install-fx82imqt/ssl-remote-sensing_5f23d22ea442468c8cc209071be0b5d8
  Running command git checkout -b gan/pipeline --track origin/gan/pipeline
  Switched to a new branch 'gan/pipeline'
  Branch 'gan/pipeline' set up to track remote branch 'gan/pipeline' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for ssl-remote-sensing: filename=ssl_remote_sensing-0.0.post1.dev118+g57dd686-py3-none-any.whl size=37536 sha256=3799491cdbdaba0e98e278c7be98b98f6fa918592398f12f9c38af6638f

In [2]:
# Log in to your W&B account
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marccgrau (unisg-ds-nlp). Use `wandb login --relogin` to force relogin


True

In [3]:
from datetime import datetime
import torch
import os
import numpy as np
from tqdm import tqdm

from torch import nn
import torchvision.transforms as T
from sklearn.metrics import classification_report

from ssl_remote_sensing.downstream_tasks.classification.model import (
    DownstreamClassificationNet,
)
from ssl_remote_sensing.constants import RANDOM_INITIALIZATION
from ssl_remote_sensing.pretext_tasks.utils import (
    load_encoder_checkpoint_from_pretext_model,
)
from ssl_remote_sensing.data.get_eurosat import get_eurosat_normalizer, get_eurosat_dataloader

/usr/local/lib/python3.7/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.7/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiveTask] = FeatureMapContr

## Dataset Loading ##

In [4]:
class RunConfig:
    def __init__(self):
        self.num_epochs = 30  # number of training epochs
        self.seed = 1234  # randomness seed
        self.save = "./saved_models/"  # save checkpoint
        self.batch_size = 256
        self.learning_rate = 1e-3
        self.embedding_size = 128  # papers value is 128
        self.test_split_ratio = 0.2
        self.checkpoint_name = None


config = RunConfig()

In [5]:
# Setup data loading
eurosat_normalizer = get_eurosat_normalizer()
train_dl, test_dl = get_eurosat_dataloader(
    root="./",
    transform=T.Compose([T.ToTensor(), eurosat_normalizer]),
    batchsize=config.batch_size,
    numworkers=os.cpu_count(),
    split=(config.test_split_ratio==0.2),
)

[LOG] Total number of images: 27000
[LOG] Batch size is 256
[LOG] Total images in the train set is: 21600
[LOG] Total number of batches in the trainloader: 85
[LOG] Total number of batches in the valloader: 22


## Model Training ##


In [6]:
# First of all, let's verify if a GPU is available on our compute machine. If not, the cpu will be used instead.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device used: {}".format(device))

# define the optimization criterion / loss function
loss_criterion = nn.CrossEntropyLoss().to(device)

Device used: cuda:0


## Setup Checkpoint Loading ##

In [7]:
from google.colab import drive

drive.mount("/content/drive")
g_drive_path = "/content/drive/MyDrive/deep_learning_checkpoints"
check_point_paths = os.listdir(g_drive_path)
check_point_paths += [RANDOM_INITIALIZATION]
check_point_paths = [g_drive_path + "/" + x for x in check_point_paths]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
check_point_paths = check_point_paths[3:6]

In [9]:
for filename in check_point_paths:
    # Update checkpoint name
    config.checkpoint_name = filename
    # Load Encoder from different pre-text architectures
    encoder = load_encoder_checkpoint_from_pretext_model(
        path_to_checkpoint=filename,
    )
    wandb.init(
        project="ssl-remote-sensing-classification",
        name=filename,
        config=config.__dict__,
    )
    # Model Setup
    if "bigan" in config.checkpoint_name.lower():
        model = DownstreamClassificationNet(input_dim=100, encoder=encoder, gan_encoder = True).to(device)
    else:
        model = DownstreamClassificationNet(input_dim=512, encoder=encoder).to(device)
    # define learning rate and optimization strategy
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    model.train()
    train_epoch_loss = np.NaN
    validation_epoch_loss = np.NaN

    with tqdm(range(config.num_epochs)) as tq:
        for epoch in tq:
            # print epoch loss
            now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            tq.desc = f"[{now}] epoch: {epoch+1} train-loss: {train_epoch_loss} validation-loss: {validation_epoch_loss}"
            # init collection of mini-batch losses
            train_mini_batch_losses = []

            # iterate over all-mini batches
            for i, (images, labels) in enumerate(train_dl):

                # push mini-batch data to computation device
                images = images.to(device)
                labels = labels.to(device)

                # forward + backward + optimize
                optimizer.zero_grad()
                out = model(images)
                loss = loss_criterion(out, labels)
                loss.backward()
                optimizer.step()

                # collect mini-batch reconstruction loss
                train_mini_batch_losses.append(loss.data.item())
                wandb.log(
                    {
                        "step/training_loss": loss.data.item(),
                    }
                )

            # determine mean min-batch loss of epoch
            train_epoch_loss = np.mean(train_mini_batch_losses)

            # Specify you are in evaluation mode
            model.eval()
            with torch.no_grad():
                validation_mini_batch_losses = []
                for (images, labels) in test_dl:
                    images = images.to(device)
                    labels = labels.to(device)
                    # calculate outputs by running images through the network
                    outputs = model(images)
                    # the class with the highest energy is what we choose as prediction
                    validation_epoch_loss = loss_criterion(outputs, labels)
                    # collect mini-batch reconstruction loss
                    validation_mini_batch_losses.append(
                        validation_epoch_loss.data.item()
                    )
                validation_epoch_loss = np.mean(validation_mini_batch_losses)

            wandb.log(
                {
                    "epoch/training_loss": train_epoch_loss,
                    "epoch/validation_loss": validation_epoch_loss,
                }
            )
    # Store classification report
    y_pred = []
    y_true = []
    # iterate over test data
    model.eval()
    with torch.no_grad():
        for (images, labels) in tqdm(test_dl, desc="Predict labels"):
            images = images.to(device)

            outputs = model(images)  # Feed Network
            _, predicted = torch.max(outputs, 1)

            y_pred.extend(predicted.cpu().numpy())  # Save Prediction
            y_true.extend(labels.numpy())  # Save Truth
    # log results
    wandb.log(
        {
            "classification_report": classification_report(
                y_true, y_pred, output_dict=True
            )
        }
    )
    print(classification_report(y_true, y_pred))

[20221119-11:30:52] epoch: 30 train-loss: 0.11780322088914759 validation-loss: 0.7217872224070809: 100%|██████████| 30/30 [07:57<00:00, 15.92s/it]
Predict labels: 100%|██████████| 22/22 [00:02<00:00,  7.53it/s]


              precision    recall  f1-score   support

           0       0.91      0.79      0.85       595
           1       0.96      0.95      0.96       599
           2       0.80      0.84      0.82       585
           3       0.59      0.72      0.65       512
           4       0.89      0.92      0.91       511
           5       0.90      0.76      0.82       405
           6       0.78      0.71      0.74       530
           7       0.91      0.96      0.93       567
           8       0.78      0.76      0.77       495
           9       0.95      0.98      0.97       601

    accuracy                           0.84      5400
   macro avg       0.85      0.84      0.84      5400
weighted avg       0.85      0.84      0.85      5400



epoch/training_loss,█▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/validation_loss,█▄▃▃▃▂▂▂▂▂▁▁▁▁▂▁▂▁▁▁▁▂▂▂▂▂▂▂▄▃
step/training_loss,█▅▄▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁
epoch/training_loss,0.12391
epoch/validation_loss,0.6118
step/training_loss,0.16098


[20221119-11:38:54] epoch: 30 train-loss: 0.15584414697745266 validation-loss: 0.5074218653819778: 100%|██████████| 30/30 [07:49<00:00, 15.64s/it]
Predict labels: 100%|██████████| 22/22 [00:02<00:00,  7.51it/s]


              precision    recall  f1-score   support

           0       0.86      0.87      0.87       595
           1       0.98      0.90      0.93       599
           2       0.72      0.85      0.78       585
           3       0.68      0.65      0.67       512
           4       0.89      0.93      0.91       511
           5       0.85      0.82      0.83       405
           6       0.81      0.66      0.73       530
           7       0.95      0.94      0.95       567
           8       0.74      0.78      0.76       495
           9       0.94      0.99      0.96       601

    accuracy                           0.84      5400
   macro avg       0.84      0.84      0.84      5400
weighted avg       0.85      0.84      0.84      5400



epoch/training_loss,█▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch/validation_loss,█▆▅▄▃▃▃▂▂▄▂▂▁▂▂▂▁▁▃▁▂▂▃▂▂▂▂▃▂▄
step/training_loss,█▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁
epoch/training_loss,0.14057
epoch/validation_loss,0.61256
step/training_loss,0.07311


[20221119-11:46:57] epoch: 30 train-loss: 0.26222710591905257 validation-loss: 0.3657087419520725: 100%|██████████| 30/30 [07:53<00:00, 15.77s/it]
Predict labels: 100%|██████████| 22/22 [00:02<00:00,  7.63it/s]

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       595
           1       0.98      0.78      0.87       599
           2       0.89      0.50      0.64       585
           3       0.88      0.80      0.84       512
           4       0.94      0.89      0.91       511
           5       0.66      0.91      0.76       405
           6       0.61      0.78      0.69       530
           7       0.96      0.98      0.97       567
           8       0.84      0.84      0.84       495
           9       0.92      0.98      0.95       601

    accuracy                           0.84      5400
   macro avg       0.85      0.84      0.84      5400
weighted avg       0.86      0.84      0.84      5400

